In [2]:
import numpy as np
import pandas as pd
import math
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.formula.api import glm
import collections
from statsmodels.sandbox.stats.multicomp import multipletests


In [3]:
df=pd.read_csv('Final_poisson.csv').iloc[:,1:]

In [4]:
import time

In [5]:
def conteo_perf(newdf=df):
    df_drop=newdf.drop_duplicates(subset='LAT_LON') 
    conteos=newdf[['LAT_LON','SITIO_EXACTO_ACCIDENTE']].groupby(['LAT_LON','SITIO_EXACTO_ACCIDENTE']).size().reset_index(name='count')
    grp_conteos = conteos.groupby('LAT_LON')['count'].agg(['sum','max'])
    grp_conteos['sum_rate'] = grp_conteos['max'] / grp_conteos['sum']
    filtered_conteos = grp_conteos[grp_conteos['sum_rate'] > 0.3]
    return_df = filtered_conteos.drop(['max', 'sum_rate'], axis=1)
    return_df.rename(columns = {'sum':'NACCIDENTS'}, inplace = True) 
    fusion=pd.merge(return_df,df_drop,on='LAT_LON',how='inner')[['NACCIDENTS','AÑO_ACCIDENTE','MES_ACCIDENTE',
                                                     'CLASE_ACCIDENTE','LONGITUD', 'LATITUD',
                                                     'POT', 'PIEZA_URBANA', 'BUS_SUBIDA','BUS_BAJADA', 
                                                     'TRAFICO','ES_FESTIVO','SOLO_HORA']].reset_index().iloc[:,1:]
    return(fusion)

start_time = time.time()
b=conteo_perf()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.07788491249084473 seconds ---


In [9]:
semaforos=pd.read_csv('lights_clean.csv')[['LATITUD','LONGITUD']]

In [24]:
print(semaforos.columns)
print(b.columns)

Index(['LATITUD', 'LONGITUD'], dtype='object')
Index(['NACCIDENTS', 'AÑO_ACCIDENTE', 'MES_ACCIDENTE', 'CLASE_ACCIDENTE',
       'LONGITUD', 'LATITUD', 'POT', 'PIEZA_URBANA', 'BUS_SUBIDA',
       'BUS_BAJADA', 'TRAFICO', 'ES_FESTIVO', 'SOLO_HORA'],
      dtype='object')


In [48]:
def distancia_auxiliar(x,lights=semaforos):
    dx = (lights['LONGITUD']-x['LONGITUD'])*40000*np.cos((lights['LATITUD']+x['LATITUD'])*math.pi/360)/360
    dy = (lights['LATITUD']-x['LATITUD'])*40000/360
    dist=np.sqrt(dx**2+dy**2)
    return(np.min(dist))
    
def distancia_semaforos(df=b,lights=semaforos):
    return(df.apply(distancia_auxiliar,axis=1))
#     return([np.min(distancia_auxiliar(x=[aux1,aux2])) for (aux1,aux2) in zip(df[['LATITUD'],df['LONGITUD']])])
#     for i in range(df.shape[0]):
#         aux=df.iloc[i,:][['LATITUD','LONGITUD']]
#         dx = (lights['LONGITUD']-aux['LONGITUD'])*40000*np.cos((lights['LATITUD']+aux['LATITUD'])*math.pi/360)/360
#         dy = (lights['LONGITUD']-aux['LATITUD'])*40000/360
#         dist=np.sqrt(dx**2+dy**2)
#         np.min(dist)

In [49]:
start_time = time.time()
a=distancia_semaforos()
print("--- %s seconds ---" % (time.time() - start_time))

--- 9.43163013458252 seconds ---


In [50]:
a

0       2.074364
1       1.507733
2       1.630849
3       1.294282
4       2.777795
          ...   
5825    2.691293
5826    2.772020
5827    2.811648
5828    2.858991
5829    2.991998
Length: 5830, dtype: float64